In [1]:
from pyscf import gto, scf, fci
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumCircuit
from matplotlib.ticker import ScalarFormatter
from scipy.optimize import minimize
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import TwoLocal
from qiskit_ionq import IonQProvider
from qiskit.quantum_info import Statevector
import os
from qiskit.quantum_info import Pauli
from qiskit_algorithms import NumPyEigensolver
from qiskit.circuit.library import HGate , SdgGate
from qiskit import transpile
from scipy.optimize import differential_evolution
from qiskit_aer import AerSimulator, AerProvider

/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def fermion_to_qubit(problem, second_q_op, mapper_name):
  if mapper_name == "JW":
    mapper = JordanWignerMapper()
  if mapper_name == "Pa":
    mapper = ParityMapper(num_particles=problem.num_particles)
  if mapper_name == "BK":
    mapper = BravyiKitaevMapper()
  qubit_op = mapper.map(second_q_op)
  return qubit_op , mapper
# 고전값 비교용 FCI 
# H2 인경우로만 해뒀고, 거리 주면 에너지 계산할 수 있음. 
class ConvergenceReached(Exception):
    pass

def hamming_distance(state1, state2):
    """Number of differing bits"""
    return sum(c1 != c2 for c1, c2 in zip(state1, state2))

cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}
# cost_func 외부에 전역 상태 저장
convergence_state = {
    "prev_cost": None,
    "streak": 0,
    "tol": 1e-4,        # ε: 변화 허용치
    "patience": 5,      # k: 연속 허용 횟수
    "trigger_stop": False,
}
def early_stop_callback(xk, convergence):
    return convergence_state["trigger_stop"]  # True 반환 시 중단됨

banned_state = []

def cost_func(parameter, second_q_op, n_ptl, ansatz, backend,k):
    global convergence_state
    global cost_history_dict 
    global repulsion
    global banned_state
    params = ansatz.parameters 
    param_dict = dict(zip(params, parameter))
    qc = ansatz.assign_parameters(param_dict)
    qc.measure_all()
    job = backend.run(qc, shots=2000)
    result = job.get_probabilities()
    all_basis = []
    all_prob = []
    for bitstring, prob in result.items():
        count_ones = bitstring.count('1')
        half = len(bitstring) // 2
        left_ones = bitstring[:half].count('1')
        right_ones = bitstring[half:].count('1')
        if count_ones == n_ptl and left_ones==right_ones :
            all_basis.append(bitstring)
            all_prob.append(prob)

    all_basis = np.array(all_basis)
    all_prob = np.array(all_prob)
    top_indices = np.argsort(-all_prob)[:k]
    probable_basis = all_basis[top_indices]
    print("most probable_basis : ", probable_basis)

    n = len(probable_basis)
    H = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            basis_bra = probable_basis[i]
            basis_ket = probable_basis[j]
            inner_product = projection_hamiltonian(second_q_op, basis_bra , basis_ket)
            H_ij = np.real(inner_product)
            H[i, j] = H_ij 
            print("<{}|H|{}> = {}".format(basis_bra,basis_ket,H_ij))
    eigvals, eigvecs = np.linalg.eig(H)
    Energy = np.min(eigvals)

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(Energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {Energy+repulsion}]")

    prev = convergence_state["prev_cost"]
    if convergence_state["streak"] >= convergence_state["patience"]:
        print("✅ 조기 수렴 조건 충족: 최적화 강제 종료")
        raise ConvergenceReached()

    if prev is not None and abs(Energy - prev) < convergence_state["tol"]:
        convergence_state["streak"] += 1
    else:
        convergence_state["streak"] = 0
    convergence_state["prev_cost"] = Energy

    if convergence_state["streak"] >= convergence_state["patience"]:
        convergence_state["trigger_stop"] = True
        print("✅ 조기 수렴 조건 충족: 최적화 중단 예정")


    return Energy


def parsing(second_q_op):
    one_terms = []
    two_terms = []
    
    for label, coeff in second_q_op.items():  # Qiskit에서 FermionicOp → (label, coeff) 리스트로 가능
        ops = label.split()
        
        # One-electron term → "+_p -_q" 형태
        if len(ops) == 2:
            p = int(ops[0][2:])
            q = int(ops[1][2:])
            if ops[0][0] == "+" and ops[1][0] == "-":
                one_terms.append( (coeff, [-q, +p]) )  # -q, +p 순서 주의
            
        # Two-electron term → "+_p +_q -_s -_r" 형태
        elif len(ops) == 4:
            p = int(ops[0][2:])
            q = int(ops[1][2:])
            s = int(ops[2][2:])
            r = int(ops[3][2:])
            if ops[0][0] == "+" and ops[1][0] == "+" and ops[2][0] == "-" and ops[3][0] == "-":
                # direct term
                two_terms.append( (coeff, [-s, -r, +q, +p]) )
                # exchange term (negative)
                two_terms.append( (-coeff, [-r, -s, +q, +p]) )
    
    return one_terms, two_terms

def apply_operator(state, operators):
    """
    Apply operator list to state.
    operators: list of int → negative = annihilation, positive = creation
    
    Returns: (new_state_str, fermionic_sign) or (None, 0) if forbidden
    """
    state_list = list(state)
    sign = 1
    
    # Apply annihilation first (right-most first)
    for op in reversed(operators):
        if op < 0:
            p = abs(op)
            if state_list[p] == '0':
                return None, 0
            sign *= (-1) ** state_list[:p].count('1')
            state_list[p] = '0'
    
    # Apply creation
    for op in operators:
        if op > 0:
            p = op
            if state_list[p] == '1':
                return None, 0
            sign *= (-1) ** state_list[:p].count('1')
            state_list[p] = '1'
    
    new_state = ''.join(state_list)
    return new_state, sign

def projection_hamiltonian(second_q_op, basis_bra, basis_ket):
    """Compute ⟨Phi_I|H|Phi_J⟩"""
    one_terms, two_terms = parsing(second_q_op)
    distance = hamming_distance(basis_bra, basis_ket)
    total = 0.0
    print(distance)
    
    # 홀수 distance → 전자 수 보존 깨짐 → 바로 0
    if distance % 2 != 0:
        return 0.0
    
    # distance > 4 → 우리가 사용하는 H는 최대 2-body → 무조건 0
    if distance > 4:
        return 0.0
    
    # distance == 0 → one + two electron terms 모두 사용
    if distance == 0:
        for coeff, operators in one_terms:
            new_state, sign = apply_operator(basis_ket, operators)
            if new_state == basis_bra:
                total += coeff * sign
        for coeff, operators in two_terms:
            new_state, sign = apply_operator(basis_ket, operators)
            if new_state == basis_bra:
                total += coeff * sign
    
    # distance == 2 → one-electron term만 contribute 가능
    elif distance == 2:
        for coeff, operators in one_terms:
            new_state, sign = apply_operator(basis_ket, operators)
            if new_state == basis_bra:
                total += coeff * sign
    
    # distance == 4 → two-electron term만 contribute 가능
    elif distance == 4:
        for coeff, operators in two_terms:
            new_state, sign = apply_operator(basis_ket, operators)
            if new_state == basis_bra:
                total += coeff * sign
    
    return total

In [3]:
basis_bra = '10110110110111'
basis_ket = '11010110111110'

hamming_distance(basis_bra, basis_ket) 

4

In [4]:
api_key = "7IMO85iuzmzAAgri4osG5BPTeCFXrKoM"
provider = IonQProvider(api_key)
backend = provider.get_backend("simulator")
#simulator_backend = provider.get_backend("qpu.forte-1")

In [5]:


# To run on hardware, select the backend with the fewest number of jobs in the queue
#service = QiskitRuntimeService(channel="ibm_quantum", token ='55ac54cb4cbfe378d8604606bacfcab38ffd165ecdb215742e64495a259fa53f0ed9e63df5b3df4489d422865a007d9cc981fcb2caffd9e76f17d94d734a8593')
#backend = service.least_busy(operational=True, simulator=False)
#backend = GenericBackendV2(num_qubits=5)

#backend = AerSimulator(method = "statevector",noise_model=None)

In [9]:
atoms = ["Li", "H"]
basis = 'sto3g'
dist = 1.595
coords = [(0,0,0), (dist,0,0)]
charge = 0
multiplicity = 1
Co_O_moleculeinfo = MoleculeInfo(atoms, coords, charge=charge, multiplicity=multiplicity)
active_transformer = ActiveSpaceTransformer(4,4)
driver = PySCFDriver.from_molecule(Co_O_moleculeinfo, basis=basis)
problem = driver.run() 
# Transformer 적용
E_problem = active_transformer.transform(problem)

# 여기는 이후, As_transformer 로 변경. 
fermionic_hamiltonian = E_problem.hamiltonian
second_q_op = fermionic_hamiltonian.second_q_op()
print(second_q_op)

Fermionic Operator
number spin orbitals=8, number terms=564
  0.8292757566159641 * ( +_0 +_0 -_0 -_0 )
+ -0.05597056741202398 * ( +_0 +_0 -_1 -_0 )
+ -0.06926596196592835 * ( +_0 +_0 -_2 -_0 )
+ -0.05597056741202398 * ( +_0 +_1 -_0 -_0 )
+ 0.18365505073749996 * ( +_0 +_1 -_1 -_0 )
+ 0.006673055578997156 * ( +_0 +_1 -_2 -_0 )
+ -0.06926596196592835 * ( +_0 +_2 -_0 -_0 )
+ 0.006673055578997156 * ( +_0 +_2 -_1 -_0 )
+ 0.1978269568342926 * ( +_0 +_2 -_2 -_0 )
+ 0.19815946457944283 * ( +_0 +_3 -_3 -_0 )
+ 0.8292757566159641 * ( +_0 +_4 -_4 -_0 )
+ -0.05597056741202398 * ( +_0 +_4 -_5 -_0 )
+ -0.06926596196592835 * ( +_0 +_4 -_6 -_0 )
+ -0.05597056741202398 * ( +_0 +_5 -_4 -_0 )
+ 0.18365505073749996 * ( +_0 +_5 -_5 -_0 )
+ 0.006673055578997156 * ( +_0 +_5 -_6 -_0 )
+ -0.06926596196592835 * ( +_0 +_6 -_4 -_0 )
+ 0.006673055578997156 * ( +_0 +_6 -_5 -_0 )
+ 0.1978269568342926 * ( +_0 +_6 -_6 -_0 )
+ 0.19815946457944283 * ( +_0 +_7 -_7 -_0 )
+ -0.05597056741202398 * ( +_0 +_0 -_0 -_1 )
+ 0.006

In [10]:
fermionic_hamiltonian = E_problem.hamiltonian
second_q_op = fermionic_hamiltonian.second_q_op()
repulsion = fermionic_hamiltonian.constants['nuclear_repulsion_energy']
hamiltonian, mapper = fermion_to_qubit(E_problem, second_q_op, "JW")
H_op = hamiltonian.to_operator()
H_matrix = H_op.data
num_qubits = hamiltonian.num_qubits
num_particles = E_problem.num_particles
num_electrons = np.sum(num_particles)
num_spatial_orbitals = E_problem.num_spatial_orbitals


init_state = HartreeFock(num_spatial_orbitals,num_particles, mapper)
ansatz = TwoLocal(num_spatial_orbitals*2, ['ry', 'rz'], 'cz', initial_state=init_state).decompose()
#ansatz = UCCSD(num_spatial_orbitals,num_particles,mapper,initial_state=init_state)

pauli_basis = hamiltonian.paulis
coeffs = hamiltonian.coeffs
num_params = ansatz.num_parameters

""""
x0 = 2 * np.pi * np.random.random(num_params)

res = minimize(
    cost_func,
    x0,
    args=(second_q_op, num_electrons, ansatz, simulator_backend, 4),
    method="cobyla",
)

"""

bounds = [(0, 2*np.pi)] * num_params  # ansatz 파라미터 수에 따라 조절

try:
    res = differential_evolution(
        func=lambda w: cost_func(w, second_q_op, num_electrons, ansatz, backend,20),
        bounds=bounds,
        callback=early_stop_callback,
        polish=True,
    )
except ConvergenceReached:
    print("🔁 최적화가 조기 수렴 조건에 따라 종료되었습니다.")

plt.plot(range(cost_history_dict["iters"]), cost_history_dict["cost_history"] + repulsion, label='E_VQE')
plt.gca().yaxis.set_major_formatter(ScalarFormatter(useMathText=False))
plt.xlabel("Iterations (#)")
plt.ylabel("Energy (Hartree)")
plt.title("Energy Calc. for each iteration")
plt.legend()
plt.grid()
plt.draw()

most probable_basis :  ['01011100' '10100110' '00110110' '11001001' '10010110' '01010101'
 '01011001' '10101100' '11001100' '10101001' '10011100' '00110101'
 '11001010' '01010110' '01011010' '01101001' '10101010' '10010101'
 '11000110' '01100011']
0
<01011100|H|01011100> = -9.337092549405341
6
<01011100|H|10100110> = 0.0
4
<01011100|H|00110110> = -0.0003584579518627597
4
<01011100|H|11001001> = 0.0
4
<01011100|H|10010110> = 0.0
2
<01011100|H|01010101> = 0.0
2
<01011100|H|01011001> = 0.0
4
<01011100|H|10101100> = 0.0
2
<01011100|H|11001100> = 0.0
6
<01011100|H|10101001> = 0.0
2
<01011100|H|10011100> = 0.0
4
<01011100|H|00110101> = 0.0
4
<01011100|H|11001010> = 0.0
2
<01011100|H|01010110> = -0.16702010714840393
2
<01011100|H|01011010> = 0.033033106228435576
4
<01011100|H|01101001> = 0.03854522570267005
6
<01011100|H|10101010> = 0.0
4
<01011100|H|10010101> = 0.0
4
<01011100|H|11000110> = 0.0
6
<01011100|H|01100011> = 0.0
6
<10100110|H|01011100> = 0.0
0
<10100110|H|10100110> = -6.929911204

KeyboardInterrupt: 